In [ ]:
import pandas as pd

import mlflow
from datasets import Dataset, load_from_disk
import torch
from transformers import (
    XLMRobertaTokenizer,
    XLMRobertaForSequenceClassification,
    TrainingArguments,
    Trainer,
    AutoConfig,
    DataCollatorWithPadding
)

import evaluate

## 1. Load test data and tokenizer

In [ ]:
model_id = "FacebookAI/xlm-roberta-base"
# Preprocessing
tokenizer = XLMRobertaTokenizer.from_pretrained(model_id)
# This function tokenizes the input text using the RoBERTa tokenizer. 
# It applies padding and truncation to ensure that all sequences have the same length (256 tokens).
def tokenize(batch):
    return tokenizer(batch["text"], padding=True, truncation=True, max_length=256)


## 2. Run model inference

### a. Load metrics

In [ ]:
accuracy = evaluate.load("accuracy")
recall = evaluate.load("recall")
precision = evaluate.load("precision")
f1 = evaluate.load("f1")
metric = evaluate.combine(["accuracy","f1","recall","precision"])

### b. Load model using Transformer lib 

In [12]:
model = XLMRobertaForSequenceClassification.from_pretrained("xml-roberta-model")
model

XLMRobertaForSequenceClassification(
  (roberta): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250002, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768,

In [14]:
# model.push_to_hub("xlm-roberta-base-claim-detection-clef21-24")

model.safetensors: 100%|██████████| 1.11G/1.11G [00:38<00:00, 28.9MB/s]


CommitInfo(commit_url='https://huggingface.co/SophieTr/xlm-roberta-base-claim-detection-clef21-24/commit/ac52276a9a4cb1d8dce269518e1da75354b35385', commit_message='Upload XLMRobertaForSequenceClassification', commit_description='', oid='ac52276a9a4cb1d8dce269518e1da75354b35385', pr_url=None, pr_revision=None, pr_num=None)

#### i. Test metric on 1 random data sample from training data

In [ ]:
inputs = tokenizer("Ekonomist Mahfi Eğilmes claimed that there are 100000 victims of the pandemic. 20% of them comes from Asia.", return_tensors="pt")
with torch.no_grad():
    logits = model(**inputs).logits
predicted_class_id = logits.argmax().item()
display(f"Predicted label: {model.config.id2label[predicted_class_id]}")
metric.compute([predicted_class_id],[1]) # predicts, labels

#### ii. Test metric on Test data

**NOTE**: TEST ON GPU will be much faster

In [ ]:
# Test batch prediction using Trainer class on Test dataset
def inference_test_set(
    output_dir="./xml-roberta-test-log",
    test_path="./data/tokenized_dataset/test"
):
    test_token = load_from_disk(test_path)
    display(test_token)
    
    # TrainingArguments
    training_args = TrainingArguments(
        output_dir=f"{output_dir}",
        per_device_eval_batch_size=32,
    )
    
    # Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
    )
    
    predicts = trainer.predict(test_token)
    predictions = np.argmax(predicts.predictions, axis=-1)
    labels = predicts.label_ids
    
    display(f"Metrics on test dataset: {metric.compute(predictions=predictions, references=labels)}")

#### iii. Test model inference on news articles


In [ ]:
test = """Iltalehden tietojen mukaan Pitäjänmäen McDonald’s-ravintolassa tapahtuneen henkirikoksen tutkintanimike on koventunut taposta murhaksi.

Rikosnimike viittaa yleensä joko erityisen raakaan tai julmaan tekotapaan tai teon suunnitelmallisuuteen. Tapauksen tutkinnanjohtaja ei kommentoinut asiaa Iltalehdelle.

21-vuotias pienen vauvan äiti sai surmansa 6. huhtikuuta. Poliisi epäilee teosta naisen entistä puolisoa, vauvan 30-vuotiasta isää.

Miehen epäillään puukottaneen naista useita kertoja. Poliisi sai tiedon hätäkeskukselta noin kello 22.15 lauantai-iltana. Nainen kuoli saamiinsa vammoihin tapahtumapaikalla.

Ennen tapahtumaa mies oli julkaissut sosiaalisessa mediassa videoita ja kirjoituksia naiseen liittyen. Videoilla hän puhui naisesta alatyyliseen ja uhkaavaan sävyyn.

– Älä erota sitä lasta musta. (...) Mä pyydän nytten nätisti, toista kertaa en, mies sanoi yhdellä videoista.

""".split("\n\n")


In [ ]:
inputs = tokenizer(test, return_tensors="pt", padding=True, truncation=True)
with torch.no_grad():
    logits = model(**inputs).logits
display(logits)
for i,t in enumerate(test):
    predicted_class_id = logits[i].argmax().item()
    print(t)
    print(logits[i], predicted_class_id, model.config.id2label[predicted_class_id])
    print()


### c. Load model using MLFlow

In [ ]:
# Load model as a PyFuncModel.
logged_model = "dbfs:/databricks/mlflow-tracking/1266941859769377/a7378c8d769e4829b5ee714d9abd095a/artifacts/model"

# Uncomment this to download MLflow model
loaded_model = mlflow.pyfunc.load_model(logged_model)

In [ ]:
predicts = loaded_model.predict(test)
predicts

### 3. Model inference API

- Preprocess input text into array of sentences
- Tokenize sentences in batches


In [9]:
import spacy
from spacy.lang.fi.examples import sentences 
import torch
from transformers import (
    XLMRobertaTokenizer,
    XLMRobertaForSequenceClassification,
)

model_id = "FacebookAI/xlm-roberta-base"

# Preprocessing
tokenizer = XLMRobertaTokenizer.from_pretrained(model_id)

# Get model 
model = XLMRobertaForSequenceClassification.from_pretrained("xml-roberta-model")

nlp = spacy.load("fi_core_news_lg")
sentencizer = nlp.add_pipe("sentencizer")

In [10]:
def get_sentence_array(doc: str):
    return [str(s) for s in nlp(doc).sents]
    
def predicts(doc: str):
    sentence_array = get_sentence_array(doc)
    inputs = tokenizer(
        sentence_array, 
        return_tensors="pt", 
        padding=True, 
        truncation=True
    )
    with torch.no_grad():
        logits = model(**inputs).logits
    predictions = [logits[i].argmax().item() for i,t in enumerate(logits)]
    return sentence_array, predictions

In [11]:
docs = """Iltalehden tietojen mukaan Pitäjänmäen McDonald’s-ravintolassa tapahtuneen henkirikoksen tutkintanimike on koventunut taposta murhaksi.

Rikosnimike viittaa yleensä joko erityisen raakaan tai julmaan tekotapaan tai teon suunnitelmallisuuteen. Tapauksen tutkinnanjohtaja ei kommentoinut asiaa Iltalehdelle.

21-vuotias pienen vauvan äiti sai surmansa 6. huhtikuuta. Poliisi epäilee teosta naisen entistä puolisoa, vauvan 30-vuotiasta isää.

Miehen epäillään puukottaneen naista useita kertoja. Poliisi sai tiedon hätäkeskukselta noin kello 22.15 lauantai-iltana. Nainen kuoli saamiinsa vammoihin tapahtumapaikalla.

Ennen tapahtumaa mies oli julkaissut sosiaalisessa mediassa videoita ja kirjoituksia naiseen liittyen. Videoilla hän puhui naisesta alatyyliseen ja uhkaavaan sävyyn.

– Älä erota sitä lasta musta. (...) Mä pyydän nytten nätisti, toista kertaa en, mies sanoi yhdellä videoista.

"""
predicts(docs)

(['Iltalehden tietojen mukaan Pitäjänmäen McDonald’s-ravintolassa tapahtuneen henkirikoksen tutkintanimike on koventunut taposta murhaksi.\n\n',
  'Rikosnimike viittaa yleensä joko erityisen raakaan tai julmaan tekotapaan tai teon suunnitelmallisuuteen.',
  'Tapauksen tutkinnanjohtaja ei kommentoinut asiaa Iltalehdelle.\n\n',
  '21-vuotias pienen vauvan äiti sai surmansa 6. huhtikuuta.',
  'Poliisi epäilee teosta naisen entistä puolisoa, vauvan 30-vuotiasta isää.\n\n',
  'Miehen epäillään puukottaneen naista useita kertoja.',
  'Poliisi sai tiedon hätäkeskukselta noin kello 22.15 lauantai-iltana.',
  'Nainen kuoli saamiinsa vammoihin tapahtumapaikalla.\n\n',
  'Ennen tapahtumaa mies oli julkaissut sosiaalisessa mediassa videoita ja kirjoituksia naiseen liittyen.',
  'Videoilla hän puhui naisesta alatyyliseen ja uhkaavaan sävyyn.\n\n',
  '– Älä erota sitä lasta musta.',
  '(...) Mä pyydän nytten nätisti, toista kertaa en, mies sanoi yhdellä videoista.\n\n'],
 [1, 0, 1, 0, 0, 1, 0, 0, 0,